In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import MySQLdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [50]:
import requests
from bs4 import BeautifulSoup

import re

In [70]:
df = pd.read_csv('한국소비자원 생필품 및 서비스 가격 정보_20220128.csv', encoding='ansi')

In [71]:
df.head()

,상품명,조사일,판매가격,판매업소,제조사,세일여부,원플러스원
0,머거본 꿀땅콩(135g),2022-01-14,3600,CU(본사),머거본,N,N
1,머거본 꿀땅콩(135g),2022-01-14,2190,롯데슈퍼안양점,머거본,N,N
2,허니버터아몬드(210g),2022-01-14,5990,롯데슈퍼김포전원점,길림양행,N,N
3,허니버터아몬드(210g),2022-01-14,5990,롯데슈퍼길동점,길림양행,N,N
4,허니버터아몬드(210g),2022-01-14,5990,롯데슈퍼구평점,길림양행,N,N


In [72]:
df[df['상품명'] == '허니버터아몬드(210g)'].groupby(['판매가격','판매업소']).min()

상품명         조사일   제조사 세일여부 원플러스원
판매가격 판매업소                                                   
3990 롯데슈퍼남가좌점     허니버터아몬드(210g)  2022-01-28  길림양행    N     N
     롯데슈퍼대구각산점    허니버터아몬드(210g)  2022-01-14  길림양행    N     N
4950 롯데슈퍼상계7동점    허니버터아몬드(210g)  2022-01-14  길림양행    N     N
     롯데슈퍼프리미엄서초점  허니버터아몬드(210g)  2022-01-14  길림양행    N     N
4990 롯데슈퍼상남점      허니버터아몬드(210g)  2022-01-14  길림양행    N     N
...                         ...         ...   ...  ...   ...
7480 이마트해운대점      허니버터아몬드(210g)  2022-01-14  길림양행  NaN   NaN
     이마트화성봉담점     허니버터아몬드(210g)  2022-01-14  길림양행  NaN   NaN
     이마트화정점       허니버터아몬드(210g)  2022-01-14  길림양행  NaN   NaN
     이마트흥덕점       허니버터아몬드(210g)  2022-01-14  길림양행  NaN   NaN
8900 신세계백화점강남점    허니버터아몬드(210g)  2022-01-28  길림양행  NaN   NaN

[597 rows x 5 columns]

In [ ]:
df = pd.read_csv('한국소비자원 생필품 및 서비스 가격 정보_20220128.csv', encoding='cp949')

In [35]:
# date타임으로 변경 후 년, 월만 남기기
# df.rename(columns={'조사일':'년/월'})
df['조사일'] = df['조사일'].apply(lambda x : x[:7])
df['조사일'] = pd.to_datetime(df['조사일'])

In [40]:
# 제조사, 세일여부, 원플러스원 drop
# df.drop(['제조사', '세일여부', '원플러스원'], axis = 1, inplace = True)

In [64]:
# unique한 상품 리스트
prod_ = df['상품명'].unique()
for pro in prod_:
    print(pro)

In [49]:
# 용량만 분리해서 따로 칼럼으로 만들기
# 일단 보류

array(['머거본 꿀땅콩(135g)', '허니버터아몬드(210g)', '백설 소면(900g)', '옛날국수 소면(900g)',
       '고향만두(1.08kg)', '비비고 왕교자(1.05kg)', '생야채돼지고기물만두(800g)',
       '백설 자른 햇당면(300g)', '옛날 자른당면(300g)', '국산콩두부 부침용(380g)',
       '국산콩두부 찌개용(380g)', '행복한콩 부침두부(300g)', '행복한콩 찌개두부(300g)',
       '삼양라면(5개입)', '신라면(5개입)', '진라면 순한맛(5개입)', '곰표 밀가루 중력다목적용(1kg)',
       '백설 찰밀가루(1kg)', '백설 5가지 재료 부침가루(1kg)', '오뚜기 부침가루(1kg)',
       '삼립호떡 미니꿀호떡(16개입)', '정통크림빵(3개입)', '보노 콘스프(3개입)',
       '보노 포르치니버섯스프(3개입)', '오뚜기 크림스프(80g)', '스페셜K 오리지널(480g)',
       '켈로그 콘푸로스트(600g)', '포스트 콘푸라이트(600g)', '오뚜기 간편 미역국(5개입)',
       '오뚜기 간편 북엇국(5개입)', '3분 쇠고기짜장(200g)', '3분 쇠고기카레(200g)',
       '오뚜기 3분 백세카레 약간 매운맛(200g)', '청정원 카레여왕 비프카레(160g)', '오뚜기밥(210g)',
       '오뚜기밥(3개입)', '오뚜기밥(6개입) ', '햇반(210g)', '햇반(3개입)', '햇반(6개입)',
       '동원 양반 쇠고기죽(287.5g)', '동원 양반 전복죽(287.5g)', '비비고 소고기죽(280g)',
       '비비고 전복죽(280g)', '농심 신라면 큰사발면(114g)', '오뚜기 참깨라면 컵(110g)',
       '팔도 왕뚜껑(110g)', '오뚜기 맛있는 쇠고기미역국밥(162g)', '오뚜기 컵밥 제육덮밥(310g)',
       '햇반 컵반 고추장 제

In [67]:
# 지점명을 따로 추출해서 새로운 칼럼에 추가


머거본 꿀땅콩(135g)
허니버터아몬드(210g)
백설 소면(900g)
옛날국수 소면(900g)
고향만두(1.08kg)
비비고 왕교자(1.05kg)
생야채돼지고기물만두(800g)
백설 자른 햇당면(300g)
옛날 자른당면(300g)
국산콩두부 부침용(380g)
국산콩두부 찌개용(380g)
행복한콩 부침두부(300g)
행복한콩 찌개두부(300g)
삼양라면(5개입)
신라면(5개입)
진라면 순한맛(5개입)
곰표 밀가루 중력다목적용(1kg)
백설 찰밀가루(1kg)
백설 5가지 재료 부침가루(1kg)
오뚜기 부침가루(1kg)
삼립호떡 미니꿀호떡(16개입)
정통크림빵(3개입)
보노 콘스프(3개입)
보노 포르치니버섯스프(3개입)
오뚜기 크림스프(80g)
스페셜K 오리지널(480g)
켈로그 콘푸로스트(600g)
포스트 콘푸라이트(600g)
오뚜기 간편 미역국(5개입)
오뚜기 간편 북엇국(5개입)
3분 쇠고기짜장(200g)
3분 쇠고기카레(200g)
오뚜기 3분 백세카레 약간 매운맛(200g)
청정원 카레여왕 비프카레(160g)
오뚜기밥(210g)
오뚜기밥(3개입)
오뚜기밥(6개입) 
햇반(210g)
햇반(3개입)
햇반(6개입)
동원 양반 쇠고기죽(287.5g)
동원 양반 전복죽(287.5g)
비비고 소고기죽(280g)
비비고 전복죽(280g)
농심 신라면 큰사발면(114g)
오뚜기 참깨라면 컵(110g)
팔도 왕뚜껑(110g)
오뚜기 맛있는 쇠고기미역국밥(162g)
오뚜기 컵밥 제육덮밥(310g)
햇반 컵반 고추장 제육덮밥(250g)
햇반 컵반 미역국밥(167g)
비비고 사골곰탕(500g)
비비고 육개장(500g)
오뚜기 옛날 사골곰탕(350g)
오뚜기 옛날 육개장(300g)
오리온 포카칩 오리지널(66g)
포테토칩 오리지날(125g)
롯데 자일리톨 오리지날(97g)
빙그레 참붕어 싸만코(150ml)
아이비(270g)
참크래커(280g)
돼지바(70ml)
메로나(75ml)
해태 바밤바(63ml)
롯데 목캔디 허브민트(217g)
오리온 통아몬드 캔디(90g)
농심 새우깡(90

In [69]:
df['판매업소'].unique()

array(['CU(본사)', '롯데슈퍼안양점', '롯데슈퍼김포전원점', '롯데슈퍼길동점', '롯데슈퍼구평점', '롯데슈퍼광혜원점',
       '롯데슈퍼교문점', '롯데슈퍼광주점', '롯데슈퍼광주도평점', '롯데슈퍼광명점', '롯데슈퍼광명소하점',
       '롯데슈퍼공릉점', '롯데슈퍼고양원흥점', '롯데슈퍼고양풍동점', '롯데슈퍼고양식사점', '롯데슈퍼고양동산점',
       '롯데슈퍼경주용황점', '롯데슈퍼경주동천점', '롯데슈퍼개화산점', '롯데슈퍼거제상동점', '롯데슈퍼갈월점',
       '롯데슈퍼가락점', '이마트계양점', '롯데슈퍼G화곡점', '롯데슈퍼G홍제점', '롯데슈퍼G철원점',
       '롯데슈퍼G춘천점', '롯데슈퍼G중동점', '롯데슈퍼G전곡점', '롯데슈퍼G은평점', '롯데슈퍼G은마점',
       '롯데슈퍼G유진점', '이마트평촌점', '이마트자양점', '롯데슈퍼수성점', '이마트월배점', '이마트여수점',
       '이마트울산점', '이마트춘천점', '롯데슈퍼연수점', '롯데슈퍼범서점', '롯데슈퍼시흥조남점', '롯데슈퍼시흥장곡점',
       '롯데슈퍼시흥능곡점', '롯데슈퍼시흥은행점', '롯데슈퍼송천점', '롯데슈퍼송정점', '롯데슈퍼송양점',
       '롯데슈퍼세류역점', '롯데슈퍼세곡점', '롯데슈퍼세교점', '롯데슈퍼성주점', '롯데슈퍼성수점', '롯데슈퍼성내점',
       '롯데슈퍼서초3동점', '롯데슈퍼서초국제점', '롯데슈퍼서창점', '롯데슈퍼상계7동점', '롯데슈퍼상계11점',
       '롯데슈퍼삼천포점', '롯데슈퍼산본2동점', '롯데슈퍼삼성역점', '롯데슈퍼분당이매점', '롯데슈퍼분당서현점',
       '롯데슈퍼분당구미점', '롯데슈퍼부산화명점', '롯데슈퍼부산학장점', '롯데슈퍼부산문현점', '롯데슈퍼부산정관점',
       '롯데슈퍼봉은사로점', '롯데슈퍼복현점', '롯데슈퍼보령동대점', '롯데슈퍼방화점', '롯데슈퍼방학점',
       '롯데슈퍼명동점', '롯데슈퍼명지점

In [ ]:
li = ['CU','롯데슈퍼','이마트','(주)부산경남유통','GS더프래시','(주)농협하나로유통','세븐일레븐','GS25']